In [337]:
#!/Tsan/bin/python
# -*- coding: utf-8 -*-

In [338]:
# Libraries To Use
from __future__ import division 
import numpy as np
import pandas as pd
import statsmodels.api as sm
import os
from sklearn import linear_model
from datetime import datetime,time,date
import matplotlib.pyplot as plt
import theano.tensor as T
from theano import function
import seaborn as sns
from theano.tensor.shared_randomstreams import RandomStreams

In [339]:
# Import My own library for factor testing
import factorFilterFunctions as ff
#from config import *

In [340]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [341]:
path = ff.data_path # path

In [342]:
# --------------------------------------- Calculate Forward Adjusted Price ---------------------------------- #

In [343]:
# the necessary files
filenameAdjustFactor='LZ_GPA_CMFTR_CUM_FACTOR.csv'
filenamePirce='LZ_GPA_QUOTE_TCLOSE.csv'

In [344]:
# first is to calculate forward adjusted pice
def calAdjustedPrice():
    # Adjusted factor
    AdjFacBackward=pd.read_csv(path+filenameAdjustFactor,infer_datetime_format=True,parse_dates=[0],index_col=0)
    AdjFacBackward=AdjFacBackward[:]

    #PriceData to Adjust
    PriceToAdj=pd.read_csv(path+filenamePirce,infer_datetime_format=True,parse_dates=[0],index_col=0)
    PriceToAdj=PriceToAdj[:]

    #Calculate
    AdjFacforward = AdjFacBackward/AdjFacBackward.max()
    adjustedPrice = (AdjFacforward*PriceToAdj).round(5)
    #adjustedPrice.index.name = 'LZ_GPA_DERI_AdjustedPriceForward_20-d' 
    adjustedPrice.index.name = 'Own_Factor_AdjustedPriceForward-1d'
    print adjustedPrice.index.name 
    adjustedPrice.to_csv(path+'Own_Factor_AdjustedPriceForward-1d.csv',na_rep='NaN',date_format='%Y%m%d')

In [345]:
calAdjustedPrice()

Own_Factor_AdjustedPriceForward-1d


In [346]:
# --------------------------------------- Calculate ILLQ Factor(5-days average) ---------------------------------- #

In [347]:
# 
filenameClose = 'LZ_GPA_QUOTE_TCLOSE.csv'
filenameOpen = 'LZ_GPA_QUOTE_TOPEN.csv'
filenameVolume = 'LZ_GPA_QUOTE_TVOLUME.csv'

In [348]:
def calcILLQ(): # rolling_window is set as 5 days
    openPrice = pd.read_csv(path+filenameOpen,infer_datetime_format=True,parse_dates=[0],index_col=0)
    closePrice = pd.read_csv(path+filenameClose,infer_datetime_format=True,parse_dates=[0],index_col=0)
    volume = pd.read_csv(path+filenameVolume,infer_datetime_format=True,parse_dates=[0],index_col=0)
    if openPrice.shape != closePrice.shape:
        print openPrice.shape, closePrice.shape
        print 'data shape is not equal!'
    else:
        newdf = np.abs((closePrice - openPrice)/openPrice)/volume
        newdf = newdf.rolling(min_periods=5,window=5,center=False).mean()
        newdf.index.name = 'Own_Factor_ILLQ-1d'
        newdf.to_csv(path+'Own_Factor_ILLQ-1d.csv',na_rep='NaN',date_format='%Y%m%d')
        return newdf           

In [349]:
#filenameILLQ = 'Own_Factor_ILLQ-1d.csv'
#openPrice = pd.read_csv(path+filenameILLQ,infer_datetime_format=True,parse_dates=[0],index_col=0)

In [350]:
openPrice

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,300637.SZ,300639.SZ,601366.SH,002861.SZ,300638.SZ,603797.SH,603050.SH,603826.SH,300604.SZ,300640.SZ
Own_Factor_ILLQ-1d,,,,,,,,,,,,,,,,,,,,,
2005-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-10,2.123191e-06,4.224169e-07,6.876944e-07,1.371171e-06,1.390142e-06,5.990377e-07,5.894226e-07,5.568735e-06,5.362653e-06,9.999281e-08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-11,1.437280e-06,3.693184e-07,6.910161e-07,1.299965e-06,1.165707e-06,5.467133e-07,6.790673e-07,5.803541e-06,5.727077e-06,1.123249e-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-12,7.233644e-07,3.097896e-07,6.374127e-07,1.178365e-06,1.020056e-06,3.608365e-07,4.733407e-07,5.132266e-06,4.842372e-06,7.643384e-08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-13,7.873558e-07,2.631270e-07,6.557110e-07,1.236543e-06,1.323846e-06,1.972224e-07,6.168110e-07,3.239173e-06,4.453290e-06,9.424528e-08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-14,1.004674e-06,2.840013e-07,6.739297e-07,1.117985e-06,1.226204e-06,3.142607e-07,6.500605e-07,8.336295e-07,4.475427e-06,1.224696e-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [351]:
calcILLQ().tail()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,603797.SH,603050.SH,603826.SH,300604.SZ,300640.SZ,002863.SZ,300641.SZ,603225.SH,603081.SH,603803.SH
Own_Factor_ILLQ-1d,,,,,,,,,,,,,,,,,,,,,
2017-04-14,8.756199e-08,5.612585e-08,NaN,1.269826e-07,1.479263e-07,6.466520e-08,NaN,NaN,NaN,2.549534e-08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-17,8.575471e-08,4.707496e-08,NaN,1.698211e-07,1.416233e-07,9.494669e-08,NaN,NaN,NaN,2.601485e-08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-18,8.025323e-08,4.795296e-08,NaN,1.623506e-07,1.791827e-07,1.334674e-07,NaN,NaN,NaN,3.033694e-08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-19,1.047747e-07,5.711793e-08,NaN,1.848977e-07,2.012672e-07,1.790448e-07,NaN,NaN,NaN,2.953498e-08,...,0.000953,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [352]:
# --------------------------------------- Calculate FCAP Adjusted Turnover Volume---------------------------------- #

In [353]:
# REMINDER: Code in this section can be used to neutralize any new factor! Use this to check some size-affected factor!!!

In [354]:
filenameTURNOVER = 'LZ_GPA_QUOTE_TVOLUME.csv'
filenameFCAP = 'LZ_GPA_VAL_A_FCAP.csv'

In [355]:
def simpleNormalize(narrowedData):
    dataWinsorized = narrowedData.copy()
    dataWinsorizedTrans = dataWinsorized.T
    MAD = 1.483*np.abs(dataWinsorizedTrans-dataWinsorizedTrans.median(skipna=True))
    return ((dataWinsorizedTrans - dataWinsorizedTrans.mean(axis=0, skipna=True))/dataWinsorizedTrans.std(axis=0, skipna=True)).T

In [356]:
TURNOVER = pd.read_csv(path+filenameTURNOVER,infer_datetime_format=True,parse_dates=[0],index_col=0)
FCAP1 = np.log10(pd.read_csv(path+filenameFCAP,infer_datetime_format=True,parse_dates=[0],index_col=0))

TURNOVER= simpleNormalize(TURNOVER)

FCAP1 = simpleNormalize(FCAP1 )

datelist = FCAP1.index.tolist()

In [357]:
def neutralizeFactor(normalizedFactorDF, normalizedLFCAPDF, datelist):
    factorNeutralized = pd.DataFrame(index=normalizedFactorDF.index, columns=normalizedFactorDF.columns, data=None, dtype = float)
    for date in datelist:
        LFCAPIndice = normalizedLFCAPDF.loc[date].dropna()
        factorIndice = normalizedFactorDF.loc[date].dropna()
        intersectionStocks = list(set(LFCAPIndice.index) & set(factorIndice.index))
        #dummy_Matrix = pd.get_dummies(IndustryDF.loc[date]).T.iloc[:-1]
        #dummy_Matrix = dummy_Matrix[intersectionStocks].append(LFCAPIndice.loc[intersectionStocks])
        try:
            result = sm. OLS(factorIndice.loc[intersectionStocks].T, LFCAPIndice.loc[intersectionStocks].T).fit()
            factorNeutralized.loc[date][intersectionStocks] = result.resid
        except:
            factorNeutralized.loc[date] = np.NaN
    return factorNeutralized

In [358]:
adjustedTurnOver = neutralizeFactor(TURNOVER , FCAP1, datelist)

In [359]:
adjustedTurnOver.index.name = 'Own_Factor_AdjustedTurnOver-1d'
adjustedTurnOver.to_csv(path+'Own_Factor_AdjustedTurnOver-1d.csv',na_rep='NaN',date_format='%Y%m%d')

In [360]:
# --------------------------------------- Calculate FCAP Adjusted PB ---------------------------------- #

In [361]:
filenamePB = 'LZ_GPA_VAL_PB.csv'
filenameFCAP = 'LZ_GPA_VAL_A_FCAP.csv'
PB = pd.read_csv(path+filenamePB ,infer_datetime_format=True,parse_dates=[0],index_col=0)
PB = simpleNormalize(PB)

In [362]:
adjustedPB = neutralizeFactor(PB , FCAP1, datelist)
adjustedPB.index.name = 'Own_Factor_AdjustedPB-1d'
adjustedPB.to_csv(path+'Own_Factor_AdjustedPB-1d.csv',na_rep='NaN',date_format='%Y%m%d')

In [363]:
pb1 = pd.read_csv(path+'Own_Factor_AdjustedPB-1d.csv',infer_datetime_format=True,parse_dates=[0],index_col=0)

In [364]:
pb1.columns

Index([u'000005.SZ', u'600601.SH', u'600602.SH', u'600651.SH', u'600652.SH',
       u'600653.SH', u'600654.SH', u'600656.SH', u'000004.SZ', u'000002.SZ',
       ...
       u'603797.SH', u'603050.SH', u'603826.SH', u'300604.SZ', u'300640.SZ',
       u'002863.SZ', u'300641.SZ', u'603225.SH', u'603081.SH', u'603803.SH'],
      dtype='object', length=3286)

In [365]:
# --------------------------------------- Calculate  x-days return volatility  ---------------------------------- #

In [366]:
filenameAdjPrice =  'Own_Factor_AdjustedPriceForward-1d.csv'

In [367]:
def calReturnVol(period): # 90days maybe good
    AdjPrice = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0)
    returnDF = AdjPrice.pct_change()
    newdf = returnDF.rolling(min_periods=20,window=period,center=False).std()
    newdf.index.name = 'Own_Factor_Volatility_%dd' % period
    newdf.to_csv(path+'Own_Factor_Volatility_%dd.csv' % period,na_rep='NaN',date_format='%Y%m%d')
    return newdf

In [368]:
calReturnVol(90).tail()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,603797.SH,603050.SH,603826.SH,300604.SZ,300640.SZ,002863.SZ,300641.SZ,603225.SH,603081.SH,603803.SH
Own_Factor_Volatility_90d,,,,,,,,,,,,,,,,,,,,,
2017-04-14,0.013380,0.013679,0.011930,0.017494,0.017943,0.012376,0.010654,NaN,0.019156,0.017770,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-17,0.013427,0.013675,0.011930,0.017723,0.018247,0.012497,0.010651,NaN,0.019123,0.017401,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-18,0.013429,0.013681,0.011926,0.017723,0.018552,0.012513,0.010599,NaN,0.019110,0.017400,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-19,0.013260,0.013667,0.011555,0.017699,0.018473,0.012578,0.010262,NaN,0.019082,0.017322,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-20,0.013329,0.013699,0.011615,0.017790,0.018576,0.012530,0.010229,NaN,0.019160,0.017052,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [369]:
# --------------------------------------- Calculate  x-days return above 20 days MA  ---------------------------------- #

In [370]:
def calAbove20MA(period):
    AdjPrice = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0)
    newdf = AdjPrice - AdjPrice.rolling(min_periods=20,window=period,center=False).mean()
    newdf = newdf.rolling(min_periods=20,window=period,center=False).mean()
    newdf.index.name = 'Own_Factor_Above20MA_%dd' % period
    newdf.to_csv(path+'Own_Factor_Above20MA_%dd.csv' % period,na_rep='NaN',date_format='%Y%m%d')
    return newdf 

In [371]:
calAbove20MA(20).tail()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,603797.SH,603050.SH,603826.SH,300604.SZ,300640.SZ,002863.SZ,300641.SZ,603225.SH,603081.SH,603803.SH
Own_Factor_Above20MA_20d,,,,,,,,,,,,,,,,,,,,,
2017-04-14,-0.075425,-0.129875,7.727152e-15,0.039000,-0.858975,-0.066925,1.953993e-15,NaN,-1.211075,0.073625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-17,-0.079300,-0.134300,7.727152e-15,-0.000075,-0.906325,-0.071300,1.953993e-15,NaN,-1.266775,0.024875,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-18,-0.083175,-0.136975,7.727152e-15,-0.031125,-0.959725,-0.076650,1.953993e-15,NaN,-1.307225,-0.036700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-19,-0.090500,-0.142350,7.727152e-15,-0.086700,-0.998050,-0.085325,1.953993e-15,NaN,-1.360325,-0.071975,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-04-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [372]:
# --------------------------------------- Calculate  ARoon  ---------------------------------- #

In [373]:
AdjPrice = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0).iloc[-1000:]
adcopy = AdjPrice.copy()

In [374]:
adcopy.iloc[:,2].head()

Own_Factor_AdjustedPriceForward-1d
2013-03-12    3.76528
2013-03-13    3.97446
2013-03-14    3.96450
2013-03-15    3.94458
2013-03-18    3.96450
Name: 600602.SH, dtype: float64

In [375]:
def calAroon(data, l=20):
    datacopy = data.copy()
    for i in range(l,len(datacopy)):
        s = datacopy.iloc[i-l:i]
        #print s
        try:
            data.iloc[i] = pd.Timedelta(s.idxmax().date() - s.idxmin().date()).days/l
        except:
            data.iloc[i] = np.NaN
    return data

In [376]:
#aroonData = adcopy.iloc[:,:10].apply(calAroon,l=20)

In [377]:
# --------------------------------------- Calculate  daily deal Amount(yuan)   ---------------------------------- #

In [378]:
filenameTVolume = 'LZ_GPA_QUOTE_TVOLUME.csv' # 成交量
filenameAdjPrice =  'Own_Factor_AdjustedPriceForward-1d.csv'

In [379]:
def calcDDA():
    turnOver = pd.read_csv(path+filenameTVolume,infer_datetime_format=True,parse_dates=[0],index_col=0)
    closePrice = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0)
    #volume = pd.read_csv(path+filenameVolume,infer_datetime_format=True,parse_dates=[0],index_col=0)
    if turnOver.shape != closePrice.shape:
        print turnOver.shape, closePrice.shape
        print 'data shape is not equal!'
    else:
        newdf = turnOver * closePrice
        #newdf = newdf.rolling(min_periods=5,window=5,center=False).mean()
        newdf.index.name = 'Own_Factor_DDA-1d'
        newdf.to_csv(path+'Own_Factor_DDA-1d.csv',na_rep='NaN',date_format='%Y%m%d')
        return newdf           

In [380]:
calcDDA()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,603797.SH,603050.SH,603826.SH,300604.SZ,300640.SZ,002863.SZ,300641.SZ,603225.SH,603081.SH,603803.SH
Own_Factor_DDA-1d,,,,,,,,,,,,,,,,,,,,,
2005-01-04,1.120256e+04,9.511504e+04,36486.408181,4.231575e+04,4.286417e+04,4.647830e+04,34848.379104,7.744337e+04,13739.751792,1.176296e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-05,1.104777e+04,1.539230e+05,77055.737351,1.189617e+05,3.607527e+04,3.645003e+04,66542.020171,2.682239e+04,24501.571164,2.070431e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-06,1.028649e+04,7.519487e+04,93782.209114,1.406231e+05,5.684882e+04,3.025549e+04,33524.838585,8.755773e+03,23666.695054,2.156412e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-07,4.150195e+04,1.162695e+05,81109.801898,5.668555e+04,5.576362e+04,3.496261e+04,62727.421697,3.152954e+04,17692.454404,2.093654e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-10,1.024396e+05,1.451684e+05,81562.183851,6.196563e+04,2.280886e+04,3.533523e+04,113085.368636,2.329403e+05,15792.835194,8.330268e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-11,2.546677e+04,7.329183e+04,66538.857640,7.007041e+04,2.777977e+04,3.069064e+04,53326.139704,4.126954e+05,51265.468366,7.055445e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-12,1.819887e+04,9.166338e+04,55987.950656,3.804263e+04,2.549850e+04,2.902652e+04,54666.832954,2.388097e+05,22423.051945,8.959669e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-13,2.032021e+04,1.246899e+05,40482.267958,1.689614e+05,2.643201e+04,3.676394e+04,77921.212387,3.383591e+05,49882.869110,1.099860e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-14,1.890111e+04,1.520970e+05,52393.353286,1.093032e+05,6.019900e+04,3.986366e+04,56831.726570,3.200100e+05,42039.785261,9.886799e+04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [381]:
filenameDDA = 'Own_Factor_DDA-1d.csv'
DDAdf = pd.read_csv(path+filenameDDA,infer_datetime_format=True,parse_dates=[0],index_col=0)

In [382]:
def DDAaverage(period):
    DDAmean = DDAdf.rolling(min_periods=20,window=period,center=False).mean()
    DDAmean.index.name = 'Own_Factor_DDA-%dd' % period
    DDAmean.to_csv(path+'Own_Factor_DDA-%dd.csv' % period, na_rep='NaN',date_format='%Y%m%d')
    return DDAmean

In [383]:
DDAaverage(20)

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,603797.SH,603050.SH,603826.SH,300604.SZ,300640.SZ,002863.SZ,300641.SZ,603225.SH,603081.SH,603803.SH
Own_Factor_DDA-20d,,,,,,,,,,,,,,,,,,,,,
2005-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [384]:
# --------------------------------------- Calculate  annual idiosyncratic volatility(daily updated)   ---------------------------------- #

In [385]:
def cal_factor_return(factor, stkreturn, factorname):
    '''
    To cal the return of the factor by group method( best(1/3) - worst(1/3))
    Output: Dataframe, the factor Return with only one value column
    Input:
    factor: Dataframe, the factor Value df, Note that shift(1) has been used before this function is used
    stkreturn:  Dataframe, the return of all stock in the market. No shift needed.
    factorname:  String, the name of the factor!
    '''
    factorReturn = pd.DataFrame(index=factor.index[1:-1], columns=[factorname], data =None , dtype =float)
    for date in factorReturn.index:
        factorSlice = factor.loc[date].dropna()
        stkreturnSlice = stkreturn.loc[date].dropna()
        intersection = list(set(factorSlice.index) & set(stkreturnSlice.index))
        factorSlice = factorSlice.loc[intersection]
        stkreturnSlice = stkreturnSlice.loc[intersection]
        
        q_min = factorSlice.quantile(0.33)
        q_max = factorSlice.quantile(0.66)
        q_min_univ = factorSlice[factorSlice<=q_min]
        q_max_univ = factorSlice[factorSlice>=q_max]
        q_min_return = (q_min_univ * stkreturnSlice.loc[q_min_univ.index]).sum() / q_min_univ.sum()
        q_max_return = (q_max_univ * stkreturnSlice.loc[q_max_univ.index]).sum() / q_max_univ.sum()
        #print q_min_return
        factorReturn.loc[date] = q_min_return - q_max_return
    return factorReturn

In [386]:
startTime =  datetime.strptime('20100101', '%Y%m%d')
endTime = datetime.strptime('20170328', '%Y%m%d')
filenameAdjPrice =  'Own_Factor_AdjustedPriceForward-1d.csv'
filenameFCAP = 'LZ_GPA_VAL_A_FCAP.csv'
filenamePB='LZ_GPA_VAL_PB.csv'  # 市净率
filenameBENCH = 'LZ_GPA_INDXQUOTE_CLOSE.csv'
ZZ500Index = '000905.SH' #   ZZ500 index code

In [387]:
pricedf = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0).loc[startTime:endTime]
FCAPdf  = pd.read_csv(path+filenameFCAP,infer_datetime_format=True,parse_dates=[0],index_col=0).loc[startTime:endTime]
PBdf = pd.read_csv(path+filenamePB,infer_datetime_format=True,parse_dates=[0],index_col=0).loc[startTime:endTime]
benchmarkdf = pd.read_csv(path+filenameBENCH,infer_datetime_format=True,parse_dates=[0],index_col=0)[ZZ500Index].loc[startTime:endTime].pct_change()
returndf = pricedf.pct_change()
FCAPdf = FCAPdf .shift(1)
PBdf = PBdf.shift(1)
FCAPdf.tail()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,603797.SH,603050.SH,603826.SH,300604.SZ,300640.SZ,002863.SZ,300641.SZ,603225.SH,603081.SH,603803.SH
LZ_GPA_VAL_A_FCAP-d,,,,,,,,,,,,,,,,,,,,,
2010-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-05,547332.0612,8.960466e+05,702778.5210,595563.4630,375419.7281,770153.7198,5.051239e+05,66555.3770,70307.4110,1.023546e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-06,566520.6643,9.305763e+05,709815.1019,606649.4426,389901.7948,768407.3395,5.096541e+05,66022.0807,70307.4110,1.000371e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-07,555555.7483,9.098585e+05,695741.9400,618967.1978,383774.7666,768407.3395,5.013487e+05,64528.8511,70307.4110,1.000371e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-08,532712.1731,8.856877e+05,667595.6163,605417.6671,376533.7333,747450.7757,4.930432e+05,63995.5548,70307.4110,9.926466e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-11,538194.6311,9.219439e+05,697501.0853,606033.5549,379875.7486,752689.9166,4.975734e+05,65062.1474,70307.4110,9.994058e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-12,531798.4301,9.495677e+05,693103.2222,614655.9835,380989.7538,749197.1560,5.134293e+05,66768.6955,70307.4110,9.829905e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-13,560124.4633,9.530206e+05,698380.6579,614655.9835,386559.7794,764914.5788,5.202247e+05,68261.9251,70307.4110,9.936122e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-14,551900.7762,9.789179e+05,696621.5127,592484.0242,374862.7256,752689.9166,5.096541e+05,68155.2659,70307.4110,9.694719e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [388]:
SizeReturn = cal_factor_return(FCAPdf, returndf , 'Size_Return')

In [389]:
SizeReturn.isnull().sum()

Size_Return    0
dtype: int64

In [390]:
PBReturn = cal_factor_return(PBdf, returndf , 'PB_Return')

In [391]:
PBReturn.isnull().sum()

PB_Return    0
dtype: int64

In [392]:
benchmarkdf = benchmarkdf.loc[PBReturn.index[0]:PBReturn.index[-1]]

In [393]:
benchmarkdf .isnull().sum()

0

In [394]:
returndf = returndf.loc[PBReturn.index[0]:PBReturn.index[-1]]

In [395]:
startOfMonthList, endOfMonthList = ff.getLastDayOfMonth(PBReturn.index)

In [396]:
len(startOfMonthList)

87

In [397]:
len(endOfMonthList)

87

In [398]:
datetuple = list(zip(startOfMonthList,endOfMonthList))

In [399]:
def calIdoVol(returnDF , factorReturnList , datetuple ,torelance = 0.05):
    '''
    function to calculate to idiosyncratic volatility.
    Output: Dataframe, The idiosyncratic volatility factor (same shape as ohter daily factor).
    Input : 
    returnDF: Dataframe, the data of the return of all stocks.
    factorReturn: List, the element is the factor-return dataframe. (Obtained by group-method so there is only one value \
    column(and one index column)
    for each dataframe).Note that this df has been shift(1) to ensure that future data is not used!
    datatuple: List, element is the tuple which is consisted of startTime and endTime. Usually zip by the startOfMonthList and endOfMonthList.
    torelance: float, to filter the Nan Value.
    '''
    startdf = pd.DataFrame()
    for i in datetuple:        
        returnDFSlice = returnDF.loc[i[0]:i[1]]
        tempdf= pd.DataFrame(index = returnDFSlice.index, columns = returnDFSlice.columns ,data = None ,dtype =float)
        newReturnSlice = returnDFSlice.loc[:,returnDFSlice.isnull().sum() < returnDFSlice.shape[1] * torelance]
        newReturnSlice = newReturnSlice.fillna(method = 'ffill').fillna(method = 'bfill')
        filterList = newReturnSlice.columns.tolist()
        mapfunction = map(lambda x: x.loc[i[0]:i[1]], factorReturnList)
        totaldf = pd.concat(mapfunction, axis=1) 
        for stk in filterList:
            result = sm.OLS(newReturnSlice[stk],totaldf).fit()
            tempdf[stk].loc[i[0]:i[1]] = np.std(result.resid) * np.sqrt(242)
        startdf = startdf.append(tempdf)
    return startdf        

In [400]:
dfList = [PBReturn,SizeReturn,benchmarkdf]

In [401]:
dd = calIdoVol(returndf , dfList,datetuple)

In [414]:
dd.index.name = 'idiosyncratic volatility'
dd.head()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,603797.SH,603050.SH,603826.SH,300604.SZ,300640.SZ,002863.SZ,300641.SZ,603225.SH,603081.SH,603803.SH
idiosyncratic volatility,,,,,,,,,,,,,,,,,,,,,
2010-01-05,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-06,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-07,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-08,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-11,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [403]:
dd.to_csv(path+'Own_Factor_Idiosyncratic_Volatility.csv', na_rep='NaN',date_format='%Y%m%d')

In [404]:
# --------------------------------------- Calculate  annual idiosyncratic volatility(daily updated)   ---------------------------------- #

In [405]:
ddc =dd.tail(100)

In [406]:
filenameIDIVOL = 'Own_Factor_Idiosyncratic_Volatility.csv'
idio = pd.read_csv(path+filenameIDIVOL,infer_datetime_format=True,parse_dates=[0],index_col=0)

In [412]:
idio.head()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,603797.SH,603050.SH,603826.SH,300604.SZ,300640.SZ,002863.SZ,300641.SZ,603225.SH,603081.SH,603803.SH
idiosyncratic volatility,,,,,,,,,,,,,,,,,,,,,
2010-01-05,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-06,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-07,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-08,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-11,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [413]:
dd.head()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,603797.SH,603050.SH,603826.SH,300604.SZ,300640.SZ,002863.SZ,300641.SZ,603225.SH,603081.SH,603803.SH
idiosyncratic volatility,,,,,,,,,,,,,,,,,,,,,
2010-01-05,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-06,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-07,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-08,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-11,0.267502,0.275782,0.183632,0.256143,0.131893,0.245651,0.192114,0.318826,0.0,0.161892,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [409]:
def adj_boxplot(factorData):
    '''To calculate  adjusted -boxplot winsorized data and then Normalize the outcome
    Output: Dataframe, the winsorized and normalized data
    Input: 
    factorData:Dataframe, raw data, can contain nan value
    '''
    copyData = factorData.copy()
    for i in copyData.index:
        temp = copyData.loc[i]
        x = temp.dropna().values
        if len(x) > 0:
            mc = sm.stats.stattools.medcouple(x)
            x.sort()
            q1 = x[int(0.25*len(x))]
            q3 = x[int(0.75*len(x))]
            iqr = q3-q1
            if mc >= 0:
                l = q1-1.5*np.exp(-3.5*mc)*iqr
                u = q3+1.5*np.exp(4*mc)*iqr
            else:
                l = q1-1.5*np.exp(-4*mc)*iqr
                u = q3+1.5*np.exp(3.5*mc)*iqr
            temp.loc[temp < l] = l
            temp.loc[temp > u] = u
            #factor_data.loc[i] = (temp-temp.mean())/temp.std()
    Trans = copyData.T
    return ((Trans  - Trans .mean(axis=0, skipna=True))/Trans .std(axis=0, skipna=True)).T

In [410]:
%mprun  addd = adj_boxplot(dd)

ERROR:root:Line magic function `%mprun` not found.


In [411]:
%lprun -f adj_boxplot adj_boxplot(idio)

In [ ]:
# --------------------------------------- Calculate  some random factor  ---------------------------------- #

In [415]:
filenameAdjPrice =  'Own_Factor_AdjustedPriceForward-1d.csv'
filenameVolume = 'LZ_GPA_QUOTE_TVOLUME.csv'
closePrice = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0)
tradVol = pd.read_csv(path+filenameVolume,infer_datetime_format=True,parse_dates=[0],index_col=0)

In [426]:
def calrandfac(pricedf, factordf):
    sgndf= np.sign(pricedf.pct_change())
    tempo = factordf * sgndf
    tempo = tempo.ewm(ignore_na=True, min_periods=5, halflife = 5).mean()
    return tempo

In [427]:
overheatVol = calrandfac(closePrice, tradVol)

In [428]:
overheatVol

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,603797.SH,603050.SH,603826.SH,300604.SZ,300640.SZ,002863.SZ,300641.SZ,603225.SH,603081.SH,603803.SH
OVER_HEAT_VOL,,,,,,,,,,,,,,,,,,,,,
2005-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-11,17362.211023,15599.700509,8196.017254,7338.990601,1224.180126,6698.553599,10590.838322,-14326.736494,3880.029943,28860.425910,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-12,10402.596662,4943.392692,3961.613465,3901.998182,-714.132059,2198.381750,3396.387402,1668.029096,2044.179410,39472.459598,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-13,11233.659633,12637.857932,1585.294374,9984.066901,970.715897,1740.174325,8768.956296,-15194.558737,3484.987477,50336.580403,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-14,6374.148763,201.505827,-598.890251,3853.326015,-2465.104131,-2078.950153,2830.837631,-27492.719639,1318.541297,24416.840946,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [429]:
overheatVol.index.name = 'OVER_HEAT_VOL'
overheatVol.to_csv(path+'Over_Heat_Volume.csv', na_rep='NaN',date_format='%Y%m%d')

In [430]:
# --------------------------------------- Calculate  skewness  ---------------------------------- #
filenameAdjPrice =  'Own_Factor_AdjustedPriceForward-1d.csv'
closePrice = pd.read_csv(path+filenameAdjPrice,infer_datetime_format=True,parse_dates=[0],index_col=0)

In [438]:
# calculate N-days skewness of the price
def calSkewness(pricedf,period):
    df = pricedf.rolling(min_periods=250,window=period,center=False).skew()
    df.index.name =  'Own_Factor_Skewness_%dd' % period
    df.to_csv(path+'Own_Factor_Skewness_%dd.csv' % period,na_rep='NaN',date_format='%Y%m%d')
    return df

In [439]:
priceSkewness = calSkewness(closePrice,250)